
# Exports

In [1]:
%load_ext autotime
%matplotlib inline

time: 568 ms (started: 2023-07-06 16:13:05 +03:00)


In [2]:
import pandas as pd
import joblib
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import re

time: 1.06 s (started: 2023-07-06 16:13:06 +03:00)


In [3]:
data =  pd.read_csv('Data/SimpleApproach/Exports/Exports2018-19CumulativeGravityHSlevelComplexity.csv')
data.dropna(inplace=True)
data

,refYear,reporterCode,partnerCode,distw_harmonic,contig,gdp_o,gdp_d,gdpcap_o,gdpcap_d,pop_o,...,wto_o,wto_d,refMonth,cmdCode,primaryValue,previous_year_primaryvalue,2HScmdCode,4HScmdCode,pci,density
0,2018,31.0,4.0,1754.0,0.0,4.711294e+07,1.805323e+07,4.740,0.486,9939.771,...,0.0,1.0,1,40310,73103.23,919459.61,4,403,0.875573,0.021209
1,2018,31.0,4.0,1754.0,0.0,4.711294e+07,1.805323e+07,4.740,0.486,9939.771,...,0.0,1.0,2,40310,68658.93,935433.80,4,403,0.875573,0.021209
2,2018,31.0,4.0,1754.0,0.0,4.711294e+07,1.805323e+07,4.740,0.486,9939.771,...,0.0,1.0,3,40310,92149.63,936389.79,4,403,0.875573,0.021209
3,2018,31.0,4.0,1754.0,0.0,4.711294e+07,1.805323e+07,4.740,0.486,9939.771,...,0.0,1.0,4,40310,77202.39,961346.94,4,403,0.875573,0.021209
4,2018,31.0,4.0,1754.0,0.0,4.711294e+07,1.805323e+07,4.740,0.486,9939.771,...,0.0,1.0,5,40310,82862.27,962905.93,4,403,0.875573,0.021209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59034836,2019,894.0,834.0,1380.0,1.0,2.330867e+07,6.113687e+07,1.305,1.054,17861.034,...,1.0,1.0,2,846291,1991.00,1991.00,84,8462,2.698500,0.030157
59034837,2019,894.0,834.0,1380.0,1.0,2.330867e+07,6.113687e+07,1.305,1.054,17861.034,...,1.0,1.0,2,846330,16922.00,16922.00,84,8463,2.289032,0.029007
59034838,2019,894.0,834.0,1380.0,1.0,2.330867e+07,6.113687e+07,1.305,1.054,17861.034,...,1.0,1.0,3,847920,79726.00,79726.00,84,8479,0.534465,0.036734
59034839,2019,894.0,834.0,1380.0,1.0,2.330867e+07,6.113687e+07,1.305,1.054,17861.034,...,1.0,1.0,8,870710,1076.00,1076.00,87,8707,3.884524,0.027991


time: 16min 29s (started: 2023-07-06 16:13:07 +03:00)


In [4]:
def MPE(Y_actual,Y_Predicted):
    mape = np.mean((Y_actual - Y_Predicted)/Y_actual)*100
    return mape

def MAPE(Y_actual,Y_Predicted):
    mpe = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mpe

data['primaryValue'] = data['primaryValue']/1000000
data['gdp_o'] = data['gdp_o']/1000000
data['gdp_d'] = data['gdp_d']/1000000
data['gdpcap_o'] = data['gdpcap_o']/1000000
data['gdpcap_d'] = data['gdpcap_d']/1000000
data['distw_harmonic'] = data['distw_harmonic']/1000

time: 3.02 s (started: 2023-07-06 16:29:36 +03:00)


## RF


In [5]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y


# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25building tree 24 of 25
building tree 25 of 25



[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 18.5min remaining: 39.4min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 19.7min remaining:  3.8min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 20.7min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    5.3s remaining:  1.0min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    6.3s remaining:    4.2s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    7.8s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   24.0s remaining:  4.6min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   26.2s remaining:   17.5s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   29.8s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.89
R2 on validation data: 0.87
Mean Squared Error: 9.36
Mean Absolute Error: 0.29
Adjusted R^2 Score: 0.87
MAPE:  221955.61252437916
MPE:  -221937.2901545814
time: 27min 45s (started: 2023-07-06 16:29:39 +03:00)


In [6]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y


# DT
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=100, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=-1)]: Done  56 out of 100 | elapsed: 41.4min remaining: 32.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 49.3min finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  56 out of 100 | elapsed:   22.3s remaining:   17.5s
[Parallel(n_jobs=48)]: Done 100 out of 100 | elapsed:   28.9s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  56 out of 100 | elapsed:  1.1min remaining:   52.9s
[Parallel(n_jobs=48)]: Done 100 out of 100 | elapsed:  1.5min finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  56 out of 100 | elapsed:   21.8s remaining:   17.1s
[Parallel(n_jobs=48)]: Done 100 out of 100 | elapsed:   28.6s finished


R2 on training data: 0.9
R2 on validation data: 0.88
Mean Squared Error: 8.72
Mean Absolute Error: 0.3
Adjusted R^2 Score: 0.88
MAPE:  253510.4615867417
MPE:  -253492.77513076708
time: 52min 28s (started: 2023-07-06 16:57:25 +03:00)


In [7]:
# +density

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 13.3min remaining: 28.2min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 13.5min remaining:  2.6min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 13.7min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    5.3s remaining:  1.0min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    6.9s remaining:    4.6s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    7.4s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   18.5s remaining:  3.5min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   20.7s remaining:   13.8s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   23.0s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.91
R2 on validation data: 0.88
Mean Squared Error: 8.61
Mean Absolute Error: 0.29
Adjusted R^2 Score: 0.88
MAPE:  213190.0828688209
MPE:  -213172.63107133095
time: 15min 8s (started: 2023-07-06 17:49:53 +03:00)


In [8]:
# +density

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 11.7min remaining: 24.9min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 11.8min remaining:  2.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 11.9min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    2.7s remaining:   31.0s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    3.0s remaining:    2.0s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    3.7s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   11.6s remaining:  2.2min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   12.8s remaining:    8.5s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   14.0s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.89
R2 on validation data: 0.87
Mean Squared Error: 9.36
Mean Absolute Error: 0.31
Adjusted R^2 Score: 0.87
MAPE:  261032.12405359605
MPE:  -261015.10460769827
time: 12min 46s (started: 2023-07-06 18:05:02 +03:00)


In [9]:
# +density+gdpcap

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 13.4min remaining: 28.5min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 13.9min remaining:  2.6min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 14.0min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.9s remaining:   45.0s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    4.3s remaining:    2.9s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    4.7s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   16.2s remaining:  3.1min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   17.3s remaining:   11.6s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   18.8s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.91
R2 on validation data: 0.88
Mean Squared Error: 8.63
Mean Absolute Error: 0.29
Adjusted R^2 Score: 0.88
MAPE:  215482.59090589476
MPE:  -215465.30358316872
time: 15min 2s (started: 2023-07-06 18:17:49 +03:00)


In [10]:
# +density+gdpcap

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d']]
y = data['primaryValue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 13.7min remaining: 29.1min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 14.0min remaining:  2.7min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 14.1min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    2.1s remaining:   23.8s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    2.4s remaining:    1.6s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    3.0s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   11.2s remaining:  2.1min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   12.6s remaining:    8.4s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   14.0s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.9
R2 on validation data: 0.87
Mean Squared Error: 9.4
Mean Absolute Error: 0.31
Adjusted R^2 Score: 0.87
MAPE:  254727.30446812412
MPE:  -254710.43722254643
time: 14min 56s (started: 2023-07-06 18:32:51 +03:00)


In [11]:
# +density+gdpcap+pci

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 15.5min remaining: 32.9min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 16.0min remaining:  3.0min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 16.2min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.3s remaining:   37.9s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    3.6s remaining:    2.4s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    4.2s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   11.4s remaining:  2.2min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   13.2s remaining:    8.8s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   14.7s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.91
R2 on validation data: 0.87
Mean Squared Error: 9.33
Mean Absolute Error: 0.29
Adjusted R^2 Score: 0.87
MAPE:  228656.0800736796
MPE:  -228638.4689070766
time: 17min 4s (started: 2023-07-06 18:47:48 +03:00)


In [12]:
# +density+gdpcap+pci

X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 15.8min remaining: 33.5min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 16.2min remaining:  3.1min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 16.3min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    2.4s remaining:   27.2s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    2.7s remaining:    1.8s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    3.1s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   10.4s remaining:  2.0min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   11.4s remaining:    7.6s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   12.8s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.88
R2 on validation data: 0.85
Mean Squared Error: 10.67
Mean Absolute Error: 0.33
Adjusted R^2 Score: 0.85
MAPE:  276854.45064711815
MPE:  -276837.465636142
time: 17min 12s (started: 2023-07-06 19:04:53 +03:00)


In [13]:
data.columns

Index(['refYear', 'reporterCode', 'partnerCode', 'distw_harmonic', 'contig',
       'gdp_o', 'gdp_d', 'gdpcap_o', 'gdpcap_d', 'pop_o', 'pop_d', 'wto_o',
       'wto_d', 'refMonth', 'cmdCode', 'primaryValue',
       'previous_year_primaryvalue', '2HScmdCode', '4HScmdCode', 'pci',
       'density'],
      dtype='object')

time: 3.64 ms (started: 2023-07-06 19:22:06 +03:00)


In [14]:
# +density+gdpcap+pci + No year

X = data[['distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, criterion='poisson', max_depth=15, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed: 10.0min remaining: 21.2min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 12.1min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 12.2min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    1.8s remaining:   20.5s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    2.1s remaining:    1.4s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    2.4s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    7.6s remaining:  1.5min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    8.9s remaining:    5.9s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    9.6s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.86
R2 on validation data: 0.84
Mean Squared Error: 11.66
Mean Absolute Error: 0.34
Adjusted R^2 Score: 0.84
MAPE:  273185.1909558737
MPE:  -273167.9149738762
time: 12min 57s (started: 2023-07-06 19:22:06 +03:00)


In [15]:
# +density+gdpcap+pci + No year

X = data[['distw_harmonic', 'gdp_o', 'gdp_d', 'density', 'refMonth', 'cmdCode','gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y
# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=15, verbose=2)
reg = reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)


r2_train = reg.score(X_train, y_train)
r2_val = reg.score(X_test, y_test)
print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2_val.round(2))

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_val, y_pred)
adj_r2 = 1 - (1-r2_score(y_test, y_pred)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Mean Squared Error:", mse.round(2))
print("Mean Absolute Error:", mae.round(2))
# print("R^2 Score:", r2.round(2))
print("Adjusted R^2 Score:", adj_r2.round(2))

index = y_test!=0
mape = MAPE(y_test[index], y_pred[index])
mpe = MPE(y_test[index], y_pred[index])


print('MAPE: ', mape)
print('MPE: ', mpe)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  9.0min remaining: 19.0min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 11.1min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 11.1min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    1.4s remaining:   15.7s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    1.6s remaining:    1.1s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    2.0s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    5.6s remaining:  1.1min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    6.8s remaining:    4.5s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    8.0s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:  

R2 on training data: 0.83
R2 on validation data: 0.82
Mean Squared Error: 13.42
Mean Absolute Error: 0.37
Adjusted R^2 Score: 0.82
MAPE:  325861.0213827883
MPE:  -325844.4180042898
time: 11min 46s (started: 2023-07-06 19:35:03 +03:00)


# Log transformations

In [16]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  5.5min remaining: 11.7min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  6.6min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  6.6min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.3s remaining:   38.2s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    3.7s remaining:    2.4s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    4.2s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   13.5s remaining:  2.6min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   14.8s remaining:    9.9s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   16.2s finished


R2 on training data: 0.15
R2 on validation data: 0.12
MSE:  64.18
MAE:  0.29
R-square:  0.12
MAPE:  10049.24
MPE:  -9975.22
time: 7min 45s (started: 2023-07-06 19:46:49 +03:00)


In [17]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  6.3min remaining: 13.3min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  7.5min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  7.9min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.7s remaining:   42.2s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    4.1s remaining:    2.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    4.6s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   14.3s remaining:  2.7min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   15.8s remaining:   10.5s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   18.8s finished


R2 on training data: 0.22
R2 on validation data: 0.18
MSE:  59.65
MAE:  0.29
R-square:  0.18
MAPE:  8674.79
MPE:  -8601.27
time: 9min 7s (started: 2023-07-06 19:54:35 +03:00)


In [18]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density', 'gdpcap_o','gdpcap_d']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  7.4min remaining: 15.8min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  8.7min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  8.8min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.7s remaining:   43.0s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    4.1s remaining:    2.8s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    4.7s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   14.7s remaining:  2.8min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   16.1s remaining:   10.8s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   18.0s finished


R2 on training data: 0.19
R2 on validation data: 0.14
MSE:  62.97
MAE:  0.29
R-square:  0.14
MAPE:  8456.78
MPE:  -8383.46
time: 10min 1s (started: 2023-07-06 20:03:43 +03:00)


In [19]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density', 'gdpcap_o','gdpcap_d','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25building tree 2 of 25
building tree 3 of 25

building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  8.4min remaining: 17.8min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 10.0min remaining:  1.9min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 10.1min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.7s remaining:   42.3s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    4.1s remaining:    2.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    4.8s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   14.7s remaining:  2.8min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   16.6s remaining:   11.1s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   18.5s finished


R2 on training data: 0.2
R2 on validation data: 0.13
MSE:  63.35
MAE:  0.29
R-square:  0.13
MAPE:  8080.9
MPE:  -8007.62
time: 11min 25s (started: 2023-07-06 20:13:44 +03:00)


In [20]:
X = data[['refYear', 'distw_harmonic', 'gdp_o', 'gdp_d', 'refMonth', 'cmdCode','density','pci']]
y = data['primaryValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

X_train_log = X_train.copy()
y_train_log = np.log(y_train)
X_test_log = X_test.copy()
y_test_log = y_test

X_train_log['gdp_o'] = np.log(X_train_log['gdp_o'])
X_train_log['gdp_d'] = np.log(X_train_log['gdp_d'])
X_train_log['distw_harmonic'] = np.log(X_train_log['distw_harmonic'])

X_test_log['gdp_o'] = np.log(X_test_log['gdp_o'])
X_test_log['gdp_d'] = np.log(X_test_log['gdp_d'])
X_test_log['distw_harmonic'] = np.log(X_test_log['distw_harmonic'])

indices = y_train_log[y_train_log ==-np.inf].index
X_train_log = X_train_log.drop(indices)
y_train_log = y_train_log[y_train_log !=-np.inf]

# RF
reg = model = RandomForestRegressor(min_samples_split=10, n_estimators=25, n_jobs=-1, random_state=0, max_depth=17, verbose=2)
reg = reg.fit(X_train_log, y_train_log)
y_pred = reg.predict(X_test_log)
y_pred_train = reg.predict(X_train_log)
y_pred = np.exp(y_pred)
y_pred_train = np.exp(y_pred_train)
y_train_log = np.exp(y_train_log)


mse = mean_squared_error(y_test_log,y_pred)
mae = mean_absolute_error(y_test_log,y_pred)
r2 = r2_score(y_test_log, y_pred)
r2_train = r2_score(y_train_log, y_pred_train)

index = y_test_log!=0
mape = MAPE(y_test_log[index], y_pred[index])
mpe = MPE(y_test_log[index], y_pred[index])

print("R2 on training data:", r2_train.round(2))
print("R2 on validation data:", r2.round(2))

print('MSE: ', mse.round(2))
print('MAE: ', mae.round(2))
print('R-square: ', r2.round(2))
print('MAPE: ', mape.round(2))
print('MPE: ', mpe.round(2))

/export/home/chouno/python-installation/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.


building tree 1 of 25
building tree 2 of 25
building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25
building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:  7.0min remaining: 14.8min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  8.0min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  8.1min finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:    3.7s remaining:   42.0s
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:    4.1s remaining:    2.7s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:    4.6s finished
[Parallel(n_jobs=25)]: Using backend ThreadingBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   2 out of  25 | elapsed:   14.8s remaining:  2.8min
[Parallel(n_jobs=25)]: Done  15 out of  25 | elapsed:   16.5s remaining:   11.0s
[Parallel(n_jobs=25)]: Done  25 out of  25 | elapsed:   18.8s finished


R2 on training data: 0.21
R2 on validation data: 0.16
MSE:  61.05
MAE:  0.29
R-square:  0.16
MAPE:  10238.34
MPE:  -10164.82
time: 9min 18s (started: 2023-07-06 20:25:09 +03:00)
